# Period Derivative Modeling: CTCV J2056-3014

Calculation of spin period derivative and constraints on white dwarf mass from accretion torque theory.

## Physical Background

The period derivative Pdot = dP/dt tells us how the WD spin is evolving:
- **Pdot < 0 (spin-up)**: The WD is gaining angular momentum, typically from accreting material that carries angular momentum from the disk
- **Pdot > 0 (spin-down)**: The WD is losing angular momentum, possibly through magnetic braking or propeller effects

For accreting WDs in CVs, spin-up is expected from the accretion torque:
$$\dot{P} \approx -\frac{P^2 \dot{M} \sqrt{GM R_m}}{2\pi I}$$

where M is WD mass, Ṁ is accretion rate, R_m is the magnetospheric radius, and I is moment of inertia. This depends on the WD mass-radius relation through both R and I.

In [20]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from astropy import units as u
from astropy import constants as const

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import matplotlib.colors as mcolors
from bokeh.models import Title, Whisker, ColumnDataSource

from plotting import set_axis_styles

output_notebook()

Loading BokehJS ...

## System Parameters and Physical Setup

From Salcedo et al., the white dwarf mass and accretion rate are constrained to:
$$0.7 M_\odot < M_{\rm WD} < 1 M_\odot$$
$$\dot{M} = (2.3 - 9.0) \times 10^{14} \, \mathrm{g \, s^{-1}}$$

The WD radius $R_{\rm WD}$ is computed from the Hamada & Salpeter (1961) mass-radius relation. I import a pre-calculated set of values

In [17]:
G = 6.674e-8 # cm^3 g^-1 s^-2
P = 29.6096 #s

M = np.linspace(0.7, 1, 100) * 1.989e33 # g
Mdot = np.linspace(2.3e14, 9e14, 30)  # g/s (Salcedo et al.)

Rm = ((G*M*P**2)/(4*np.pi**2))**(1/3) # Set equal to corotation radius
Rwd = np.load('../wd_mass_radius/radii.npy') #cm
I = np.load('../wd_mass_radius/inertia.npy') # g cm^2

## Expected $\dot{P}$ from Accretion Torque

### Derivation
The period derivative $\dot{P}$ can be expressed as a function of the applied torque. Starting from the angular frequency $\Omega = 2\pi/P$:

$$\dot{P} = \frac{-2\pi \dot{\Omega}}{\Omega^2} = \frac{-2\pi \tau_{\rm acc}}{I \Omega^2}$$

where $I$ is the WD moment of inertia and $\tau_{\rm acc}$ is the accretion torque.

Material accreting from the disk transfers angular momentum to the WD at the magnetospheric radius $R_m$:

$$\tau_{\rm acc} = \dot{M} R_m^2 \Omega$$

For a non-magnetic CV (nmCV), we assume spin equilibrium where the magnetospheric radius equals the corotation radius:

$$R_m \approx R_{\rm co} = \left( \frac{G M_{\rm WD} P_s^2}{4\pi^2} \right)^{1/3}$$

At this radius, material orbiting in the disk has the same angular velocity as the WD surface.

### Combined Expression
Substituting the accretion torque into the $\dot{P}$ equation:

$$\dot{P} = \frac{-2\pi \cdot \dot{M} R_m^2 \Omega}{I \Omega^2} = \frac{-\dot{M} R_m^2 P}{I}$$

where we used $\Omega = 2\pi/P$.

In [22]:
Pdots = []
for rate in Mdot:
  Pdot = (-rate*Rm**2*P)/I 
  Pdots.append(Pdot)

p = figure(width=800, height=450, 
           title=Title(text = f"Expected Ṗ from Accretion", text_font_size = "1.5em", align = "center"),
           x_axis_label='Mass (M☉)', 
           y_axis_label='Absolute Period derivative (s/s)',
           min_border = 50,
           #toolbar_location=None
         )
set_axis_styles(p)

colors = ['#' + ''.join([hex(int(255*x))[2:].zfill(2) for x in rgb[:3]]) 
         for rgb in mcolors.LinearSegmentedColormap.from_list('custom', ['blue', 'red'])(np.linspace(0, 1, len(Pdots)))]
for i, Pdot in enumerate(Pdots):
    if(i == 0 or i == len(Pdots)-1):
        p.line(M/1.989e33, np.abs(Pdot), line_color=colors[i], line_width=1.5, legend_label=f'Ṁ = {Mdot[i]:.2e} g/s') # Low mdot is blue, high mdot is red
    else: 
        p.line(M/1.989e33, np.abs(Pdot), line_color=colors[i], line_width=1.5)

p.legend.location = "top_left"
p.legend.label_text_font_size = "10pt"
show(p)

## Expected B-field under nmCV Assumption

Under the non-magnetic CV (nmCV) assumption, we assume the WD is in spin equilibrium, meaning the magnetospheric radius $R_m$ equals the corotation radius $R_{\rm co}$. This allows us to infer the magnetic field strength from the observed spin period and accretion rate.

The magnetic field strength at the WD surface can be derived from the balance between magnetic pressure and ram pressure at the magnetospheric radius. For a dipole field geometry:

$$B = \frac{\sqrt{32 \dot{M} \sqrt{G M_{\rm WD} R_{\rm WD}^7}}}{R_{\rm WD}^3}$$

- **$R_{\rm WD}$** is obtained from the Hamada & Salpeter mass-radius relation
- **$B$** represents the surface dipole field strength inferred from spin equilibrium
- The blue lines (lower $\dot{M}$) give lower B-field estimates, while red lines (higher $\dot{M}$) give higher estimates
- These B-field values (~2000-7000 G) are characteristic of non-magnetic or weakly magnetic CVs

In [24]:
mag_fields = []
for rate in Mdot:
    mag_field = np.sqrt(32*rate*np.sqrt(G*M/pow(1/Rwd,7)))/(Rwd*Rwd*Rwd)
    mag_fields.append(mag_field)

p = figure(width=800, height=450, 
           title=Title(text = f"B-field under nmCV assumption", text_font_size = "1.5em", align = "center"),
           x_axis_label='Mass (M☉)', 
           y_axis_label='B-field (G)',
           min_border = 50,
           #toolbar_location=None
         )

for i, bfield in enumerate(mag_fields):
    if(i == 0 or i == len(mag_fields)-1):
        p.line(M/1.989e33, np.abs(bfield), line_color=colors[i], legend_label=f'Ṁ = {Mdot[i]:.2e} g/s') # Low mdot is blue, high mdot is red
    else: 
        p.line(M/1.989e33, np.abs(bfield), line_color=colors[i])
set_axis_styles(p)

p.legend.location = "top_left"
show(p)
